### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [ ]:
# poetry add python-dotenv langchain langchain-openai

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_w


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="openai/gpt-oss-120b",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 대규모 텍스트 데이터를 이용해 다음에 올 단어를 예측하도록 훈련된 트랜스포머 기반 언어 모델입니다. 이 과정에서 모델은 '
 '문맥을 이해하고 의미를 파악하기 위해 수백억 개의 파라미터를 조정하며, 손실 함수(예: 교차 엔트로피)를 최소화하도록 최적화됩니다. '
 '최종적으로, 학습된 파라미터를 바탕으로 사용자의 입력에 대해 가장 가능성이 높은 답변을 생성합니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
#combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('**ChatGPT 모델의 학습 원리 (3문장)**  \n'
 '1. 방대한 텍스트 데이터를 토큰 단위로 분할해, 앞뒤 문맥을 예측하는 ‘자기 지도 학습(unsupervised '
 'pre‑training)’을 수행합니다.  \n'
 '2. 학습 과정에서는 트랜스포머 아키텍처의 다중‑헤드 어텐션을 이용해 단어 간 장기 의존성을 효율적으로 파악합니다.  \n'
 '3. 사전 학습된 가중치를 기반으로 특정 작업(예: 질문‑응답, 번역)용 데이터셋으로 ‘지도 학습(fine‑tuning)’하거나, '
 '프롬프트와 샘플링 기법으로 즉시 활용합니다.  \n'
 '\n'
 '---\n'
 '\n'
 '### ChatGPT 모델의 장점 요약  \n'
 '\n'
 '| 장점 | 설명 |\n'
 '|------|------|\n'
 '| **범용성** | 다양한 언어와 주제에 대해 자연스러운 대화를 생성할 수 있어, 챗봇·문서 요약·코드 작성 등 여러 분야에 적용 가능 '
 '|\n'
 '| **문맥 이해** | 긴 텍스트에서도 앞뒤 관계를 파악해 일관된 답변을 제공, 복잡한 질문에도 논리적인 흐름 유지 |\n'
 '| **지속적 개선** | 대규모 데이터와 최신 아키텍처를 활용해 지속적으로 성능이 향상되며, 사용자 피드백을 반영한 업데이트가 가능 '
 '|\n'
 '| **다양한 출력 제어** | 온도·탑‑P·프롬프트 엔지니어링 등 파라미터 조절을 통해 창의성·정확성·길이 등을 자유롭게 조정 가능 '
 '|\n'
 

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [5]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "claude", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT-4는 대규모 텍스트 데이터를 이용해 자기 지도 학습(self‑supervised learning) 방식으로 사전 '
 '학습(pre‑training)됩니다. 이 과정에서 모델은 입력 토큰 시퀀스를 보고 다음에 올 토큰을 예측하도록 최적화되며, 이를 통해 '
 '언어 구조와 의미를 내부적으로 파악하게 됩니다. 이후 특정 작업에 맞추어 작은 규모의 라벨 데이터로 미세 '
 '조정(fine‑tuning)하거나 인간 피드백을 활용한 강화 학습(RLHF)을 적용해 실제 사용 환경에 맞는 성능을 향상시킵니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini 모델은 방대한 텍스트와 이미지·음성 등 멀티모달 데이터를 이용해 Transformer 기반의 대규모 '
 '사전학습(pre‑training)을 진행합니다.  \n'
 '사전학습 단계에서는 다음 토큰을 예측하거나 마스크된 부분을 복원하는 자기지도(self‑supervised) 목표를 통해 언어와 시각 '
 '정보를 동시에 학습합니다.  \n'
 '그 후, 인간이 만든 지시문과 대화 데이터를 활용한 지도학습(fine‑tuning)으로 다양한 작업에 대한 응답 능력을 '
 '향상시킵니다.  \n'
 '마지막으로, 인간 피드백을 기반으로 한 강화학습(RLHF)을 적용해 실제 사용 환경에서 더 안전하고 유용한 행동을 하도록 최적화합니다.')
<class 'str'> claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude 모델은 대규모 텍스트 데이터를 기반으로 **자기지도학습**(self‑supervised l

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [7]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
## ChatGPT 모델이 어떻게 학습되는지 (한국어 상세 설명)

아래에서는 **ChatGPT**(또는 일반적인 GPT‑계열 대형 언어 모델)의 학습 과정을 크게 **세 단계**로 나누어 설명합니다.

---

## 1️⃣ 사전 학습 (Pre‑training) – “거대한 텍스트를 읽는 단계”

| 단계 | 핵심 내용 | 왜 필요한가? |
|------|-----------|--------------|
| **데이터 수집** | 웹 페이지, 책, 논문, 뉴스, 위키피디아 등 수백 억 토큰 규모의 텍스트를 크롤링·정제 | 모델이 다양한 언어·도메인 지식을 습득하도록 함 |
| **토크나이징** | 텍스트 → **토큰**(보통 BPE/Byte‑Pair Encoding) <br>예: “ChatGPT는” → `[Chat][##GPT][##는]` | 어휘 크기를 수천~수만 개 수준으로 제한하면서도 희귀어·신조어를 처리 가능 |
| **Transformer 아키텍처** | - **입력 임베딩** + **포지션 임베딩** <br>- **다중 헤드 셀프‑어텐션** (Self‑Attention) <br>- **피드‑포워드 레이어** (FFN) <br>- **잔차 연결 + 레이어 정규화** | 셀프‑어텐션은 문맥 전체를 한 번에 바라볼 수 있어, 긴 문장·문서에서도 장기 의존성을 학습 |
| **학습 목표** (

In [8]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
## ChatGPT 모델이 학습되는 원리  

아래에서는 ChatGPT(또는 일반적인 대형 언어 모델)가 어떻게 학습되는지를 **전처리 → 사전학습 → 파인튜닝 → 강화학습**의 흐름으로 나누어 단계별로 설명합니다.  

---

### 1️⃣ 데이터 전처리 (Pre‑processing)

| 단계 | 내용 | 왜 필요한가? |
|------|------|--------------|
| **텍스트 수집** | 웹 페이지, 책, 논문, 대화 로그 등 다양한 출처에서 수십억 단어를 모음 | 모델이 **다양한 언어·도메인**을 이해하도록 하기 위함 |
| **정제(Cleaning)** | HTML 태그 제거, 중복 문장 삭제, 비속어·민감 정보 필터링 등 | 학습 효율을 높이고, 불필요하거나 위험한 데이터가 모델에 들어가지 않게 함 |
| **문장 토큰화** | 텍스트를 **토큰**(보통 서브워드 단위, 예: BPE, WordPiece)으로 변환 | 모델이 고정된 어휘 집합(보통 30k~50k 토큰)으로 모든 문자열을 표현할 수 있게 함 |
| **시퀀스 길이 맞추기** | 일정 길이(예: 2048 토큰)로 패딩하거나 잘라서 배치에 맞춤 | GPU/TPU 메모리 효율을 최적화하고, 배치 연산을 가능하게 함 |

---

### 2️⃣ 사전학습 (Pre‑training) – **자기지도 학습 (Self‑Supervised Learning)**  

#### 2.1 모델 구조: 트랜스포머(Transformer)  
- **Encoder‑Decoder** 구조 중 **Decoder‑only** (GPT 시리즈) 사용  
- 기본 블록: **멀티‑헤드 셀프‑어텐션** + **피드‑포워드 네트워크**  
- **잔차 연결 (Residual)** + **Layer Normalization** 으로 안정적인 학습  

#### 2.2 학습 목표  
- **다음 토큰 예측 (Next‑Token Prediction)**:  
  \[
  \max_{\the

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [ ]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [ ]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [19]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계에는 8개의 행성이 있습니다. 

1.  수성: 태양과 가장 가까운 행성으로, 표면이 암석으로 구성되어 있고 극도로 높은 온도와 낮은 온도가 반복됩니다.
2.  금성: 태양계에서 두 번째로 가까운 행성으로, 두꺼운 대기로 인해 극심한 온실 효과가 발생하여 매우 뜨겁습니다.
3.  지구: 우리가 사는 행성으로, 물과 대기가 있어 생명체가 존재할 수 있습니다.
4.  화성: 태양계에서 네 번째로 가까운 행성으로, 붉은색의 모래사막으로 덮여 있고 물과 생명체의 존재 가능성이 있습니다.
5.  목성: 태양계에서 가장 큰 행성으로, 가스 거인이며 강력한 자기장과 수많은 위성을 가지고 있습니다.
6.  토성: 태양계에서 두 번째로 큰 행성으로, 가스 거인이며 아름다운 고리를 가지고 있습니다.
7.  천왕성: 태양계에서 일곱 번째로 가까운 행성으로, 가스 거인이며 자전축이 기울어져 있어 극단적인 기후 변화를 경험합니다.
8.  해왕성: 태양계에서 가장 먼 행성으로, 가스 거인이며 강한 바람과 극적인 기후 변화를 가지고 있습니다.

이러한 행성들은 각각 고유한 특징과 성질을 가지고 있으며, 태양계에서 중요한 역할을 합니다.


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [12]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(llm)
print(chain)

# 테스트 실행
result = chain.invoke({"input": "양자컴퓨터 정리해 주세요."})
print(result.content)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000022F394DF3B0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000022F391EDBB0> root_client=<openai.OpenAI object at 0x0000022F3651A360> root_async_client=<openai.AsyncOpenAI object at 0x0000022F3900CAA0> model_name='openai/gpt-oss-120b' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [ ]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season_name = get_current_season("north")  # 계절 값 얻기
print(f"현재 계절: {season_name}")


현재 계절: 가을


In [14]:

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
# llm = ChatOpenAI(
#     #api_key=OPENAI_API_KEY,
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
#     temperature=0.0
# )

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season_name}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")


 가을에 발생하는 자연 현상:
### 가을에 주로 나타나는 대표적인 지구과학 현상 3가지  

| 현상 | 간단한 설명 |
|------|-------------|
| **1. 낙엽 색 변화·낙엽 (Leaf Senescence)** | 가을이 되면 낮이 짧아지고 기온이 낮아지면서 식물은 광합성에 필요한 엽록소를 분해한다. 이때 잎에 있던 카로티노이드·안토시아닌 색소가 드러나면서 빨강·노랑·주황색으로 물들고, 결국 잎이 떨어진다. 이는 **식물 생태·기후 변화를 기록하는 중요한 지표**가 된다. |
| **2. 기온 역전 현상 (Temperature Inversion)** | 가을 밤에는 땅이 급격히 식으면서 지표면 가까이의 공기가 차가워지고, 위쪽 고도에서는 아직 따뜻한 공기가 남는다. 이렇게 **따뜻한 공기가 차가운 공기 위에 얹히는 현상**이 발생하면, 대기 중 오염물질·수증기가 하층에 머물게 되어 안개·스모그가 자주 나타난다. 특히 산골이나 계곡 지역에서 흔히 관측된다. |
| **3. 제트기류 남하·중위도 저기압 활동 증가** | 가을이 되면 **극지방과 열대 지방 사이의 온도 차이가 커지면서 제트기류가 남쪽(중위도)으로 이동**한다. 이로 인해 중위도 지역에 저기압(저기압성 저기압)과 전선이 자주 통과해 **강한 바람·폭우·한랭 전선**이 나타난다. 한국·동아시아에서는 가을철 장마가 끝난 뒤에도 한두 차례 “가을 폭풍”이 찾아오는 것이 이 메커니즘과 관련된다. |

> **요약**  
> - **낙엽 색 변화**는 식물 내부 화학 변화가 기후와 연동된 현상.  
> - **기온 역전**은 지표면 냉각으로 인한 대기 안정화 현상으로 안개·스모그를 유발.  
> - **제트기류 남하**와 그에 따른 **중위도 저기압·폭풍**은 가을 대기의 전반적인 흐름 변화를 보여준다.  

이 세 가지 현상은 가을에 특히 뚜렷하게 나타나며, 기후·환경 모니터링 및 재해 대비에 중요한 역할을 합니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [15]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ### 1 USD = 1 377.98 KRW 환율에 대한 종합 분석  

---

## 1. 현재 환율이 의미하는 바

| 구분 | 내용 |
|------|------|
| **명목 환율** | 1 USD = 1 377.98 KRW (2025‑09‑18 기준) |
| **전년 동월 대비** | 약 +3 ~ 4 % 상승 (2024‑09 기준 1 332 ~ 1 350 KRW 수준) |
| **주요 요인** | - 미국 연방준비제도(Fed)의 금리 인상 지속 <br> - 한국은행(BOK)의 금리 인하·완화 기대감 <br> - 글로벌 위험 회피 심리(미·중 갈등, 유가 변동) <br> - 원달러 스와프·외환시장 개입 가능성 감소 |

> **핵심 포인트**: 현재 원화는 **달러 대비 약 3–4 % 절하**된 상태이며, 이는 2023년 말부터 이어진 ‘달러 강세 흐름’이 지속되고 있음을 보여줍니다.

---

## 2. 역사적 흐름과 비교

| 연도 | 평균 환율 (KRW/USD) | 주요 사건 |
|------|-------------------|-----------|
| 2020 | 1 180 ~ 1 210 | 코로나19 초기, 달러 약세 |
| 2021 | 1 150 ~ 1 200 | 경기 회복, 달러 약세 지속 |
| 2022 | 1 200 ~ 1 300 | 러시아-우크라이나 전쟁, 달러 급등 |
| 2023 | 1 300 ~ 1 350 | Fed 연속 금리 인상, 원화 지속 약세 |
| 2024 | 1 330 ~ 1 350 | 금리 인상 둔화, 원화 회복 시도 |
| **2025** | **1 378** (현재) | **달러 강세 지속, 원화 추가 약세** |

* **장기 평균**(2000‑2025) ≈ 1 150 KRW/USD  
* 현재 수준은 **역사적 평균보다 약 20 % 이상 고평가**된 상황이며, 2008년 금융위기 시